Trying to use Just Pandas in this file

In [11]:
import numpy as np
import pandas as ps
# import dask.dataframe as ps
import math
import sys
import datetime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Start of working on Dataset

In [12]:
# df = spark.read.options(delimiter='\t', header=False).csv("/content/drive/My Drive/Colab Notebooks/Datasets/tiny.data")
# 100K
df = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/u.data", sep="\t", names=['userId','itemId','rating','timestamp'])
# 1M
# df = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/ratings.dat", sep="::", names=['userId','itemId','rating','timestamp'])
df.drop('timestamp', inplace=True, axis=1)

print(df)
print(type(df))
print(df.dtypes)
df.count()

       userId  itemId  rating
0         196     242       3
1         186     302       3
2          22     377       1
3         244      51       2
4         166     346       1
...       ...     ...     ...
99995     880     476       3
99996     716     204       5
99997     276    1090       1
99998      13     225       2
99999      12     203       3

[100000 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
userId    int64
itemId    int64
rating    int64
dtype: object


userId    100000
itemId    100000
rating    100000
dtype: int64

In [13]:
# Converting userId and itemId to integer
df[['userId', 'itemId', 'rating']] = df[['userId', 'itemId', 'rating']].astype(int)
print(type(df))
print(df.dtypes)

<class 'pandas.core.frame.DataFrame'>
userId    int64
itemId    int64
rating    int64
dtype: object


In [14]:
def randomMatrix(dimension1, dimenstion2, rank):
    return ps.DataFrame(abs(np.random.normal(scale=1./rank,size=(dimension1,dimenstion2))))

def get_matrix_dimension(df):
  # This function get max user)id and itam_id as reslult, V and H matrix dimension
  userMax = df['userId'].max()
  itemMax = df['itemId'].max()
  userMax += 1
  itemMax += 1
  return userMax, itemMax

def sub_V_WH(WH):
  WH['R'] = WH['rating'] - WH['WH']
  R = WH.drop(['rating', 'WH'], axis=1)
  return R

def mult_WH(W, H, WH, rank):
  WH = WH.merge(W,on='userId').merge(H,on='itemId')
  # print(WH)
  colst = ''
  colList = []
  for k in range (rank):
    colst += f"WH['WC{k}'] * WH['HC{k}'] + "
    colList.append(f"WC{k}")
    colList.append(f"HC{k}")
  colst = colst[0:len(colst)-2]
  print(colst)
  # print(colList)
  WH['WH'] = eval(colst)
  WH = WH.drop(colList, axis=1)
  return WH

In [15]:
# Split Dataframe to Train and Test
num_split = 1000000
train_data = df.iloc[:num_split,:]
test_data = df.iloc[num_split:,:]
# print(train_data)
# print(test_data)

In [16]:
def mult_Wk_Hk(W, H, WkHk, k):
  # print("mult_Wk_Hk started...")
  # query = f"select userId, itemId, rating, WC{k} * HC{k} as WH from WkHk, W, H where W.id = WkHk.userId and H.id = WkHk.itemId"
  WkHk = WkHk.merge(W[[f'WC{k}','userId']],on='userId').merge(H[[f'HC{k}','itemId']],on='itemId')
  # print(WkHk)
  WkHk['WkHk'] = WkHk[f'WC{k}'] * WkHk[f'HC{k}']
  WkHk = WkHk.drop([f'WC{k}', f'HC{k}'], axis=1)
  # print(WkHk)
  # print("mult_Wk_Hk ended!")
  return WkHk

def R_add_WkHk(R, WkHk):
  # print(R)
  # print(WkHk)
  WkHk = WkHk.merge(R,on=['userId', 'itemId'])
  WkHk['Rhat'] = WkHk['R'] + WkHk['WkHk']
  Rhat = WkHk.drop(['rating', 'WkHk', 'R'], axis=1)
  # print(Rhat)
  return Rhat
  # Rhat = spark1.sql("SELECT int(R.userId), int(R.itemId), (float(R.R) + float(WkHk.WH)) as Rhat  FROM R, WkHk where R.itemId = WkHk.itemId and R.userId = WkHk.userId")

def Rhat_minus_WkHk(Rhat, WkHk):
  # print("Rhat_minus_WkHk started...")
  WkHk = WkHk.merge(Rhat,on=['userId', 'itemId'])
  WkHk['R'] = WkHk['Rhat'] - WkHk['WkHk']
  R = WkHk.drop(['rating', 'WkHk', 'Rhat'], axis=1)
  # print(R)
  # print("Rhat_minus_WkHk ended!")
  return R
  # R = spark1.sql("SELECT int(Rhat.userId), int(Rhat.itemId), (float(Rhat.Rhat) - float(WkHk.WH)) as R FROM Rhat, WkHk where Rhat.itemId = WkHk.itemId and Rhat.userId = WkHk.userId")

def Update_W_H(W, H, Rhat, k, lamda):
  # print(f" k = {k}")
  # print(Rhat)  
  # print("Update_W_H started...")
  # print("Start Select aggregation...")
  # query = f"SELECT Rhat.userId, sum(Rhat.Rhat * H.HC0) / (sum(power(H.HC0,2))+{lamda}) as value FROM Rhat, H where Rhat.itemId = H.id group by Rhat.userId"

  WkHk = Rhat.merge(H[[f'HC{k}','itemId']],on='itemId')
  WkHk['s'] = WkHk['Rhat'] * WkHk[f'HC{k}']
  WkHk['m'] = WkHk[f'HC{k}'] ** 2
  # print(WkHk)
  sums = WkHk.groupby('userId').agg({'s': 'sum'}).reset_index(level=0)
  summ = WkHk.groupby('userId').agg({'m': 'sum'}).reset_index(level=0)
  new_W_ik = sums.merge(summ,on='userId')
  new_W_ik[f'WC{k}'] = new_W_ik['s'] / (new_W_ik['m'] + lamda)
  new_W_ik = new_W_ik.drop(['s', 'm'], axis=1)
  
  # query = f"SELECT Rhat.itemId, sum(Rhat.Rhat * W.WC0) / (sum(power(W.WC0,2))+{lamda}) as value FROM W join Rhat on W.id = Rhat.userId group by Rhat.itemId"
  WkHk = Rhat.merge(W[[f'WC{k}','userId']],on='userId')
  WkHk['s'] = WkHk['Rhat'] * WkHk[f'WC{k}']
  WkHk['m'] = WkHk[f'WC{k}'] ** 2 
  # print(WkHk)
  sums = WkHk.groupby('itemId').agg({'s': 'sum'}).reset_index(level=0)
  summ = WkHk.groupby('itemId').agg({'m': 'sum'}).reset_index(level=0)
  new_H_ik = sums.merge(summ,on='itemId')
  new_H_ik[f'HC{k}'] = new_H_ik['s'] / (new_H_ik['m'] + lamda)
  new_H_ik = new_H_ik.drop(['s', 'm'], axis=1)

  # print(W)
  W = W.drop(f'WC{k}', axis=1)
  W = W.merge(new_W_ik, on = 'userId')
  # print(new_W_ik)
  # print(W)

  # print(H)
  H = H.drop(f'HC{k}', axis=1)
  H = H.merge(new_H_ik, on = 'itemId')
  # print(new_H_ik)
  # print(H)
  return W, H

def ccd_iteration(W, H, R, k):
  for rank in range(k):
    WkHk = mult_Wk_Hk(W, H, V, rank)
    # print(WkHk)
    # print("WkHk calculated For Rhat.")
    Rhat = R_add_WkHk(R, WkHk)
    # print(Rhat)
    # print("Rhat calculated.")
    W, H = Update_W_H(W, H, Rhat, rank, lamda)
    # print(W)
    # print(H)
    # sys.exit()
    # print("W, H  Updated on k.")
    WkHk = mult_Wk_Hk(W, H, V, rank)
    # print(WkHk)
    # print("WkHk calculated for R.")
    R = Rhat_minus_WkHk(Rhat, WkHk)
    # print(R)
    # print("R calculated.")
  return W, H, R

def rmse_WH_V(WH):
  WH['diff'] = (WH['rating'] - WH['WH']) ** 2
  rmse = math.sqrt(WH['diff'].mean())
  return rmse


In [21]:
# from numpy import genfromtxt
k=10
lamda=60
print('rank: \t',k,'')
print('lamda: \t',lamda,'\n')

userMax, itemMax = get_matrix_dimension(train_data)
print(f"{userMax}, {itemMax}")

W = randomMatrix(userMax, k, k)
H = randomMatrix(k, itemMax, k)
H = H.T

W = W.reset_index(level=0)
H = H.reset_index(level=0)
WcolDic = {"index": "userId"}
HcolDic = {"index": "itemId"}
for rank in range (k):
  WcolDic[rank] = f"WC{rank}"
  HcolDic[rank] = f"HC{rank}"
# print(WcolDic)
# print(HcolDic)
W = W.rename(columns=WcolDic, errors="raise")
H = H.rename(columns=HcolDic, errors="raise")
tiny=False
if tiny==True:
  # Save to disk
  # H.write.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Htiny.data")
  # W.write.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Wtiny.data")
  # Read from disk
  # df = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/tiny.data", sep="\t", names=['userId','itemId','rating','timestamp'])
  W = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/CWtiny.data", sep="\t")
  H = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/CHtiny.data", sep="\t")
  # W = ps.read_csv(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Wtiny.data")
  # H = ps.read_csv(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Htiny.data")
  W = W.rename(columns={"id": "userId", "WC0": "WC0", "WC1": "WC1"}, errors="raise")
  H = H.rename(columns={"id": "itemId", "HC0": "HC0", "HC1": "HC1"}, errors="raise")
  # H = H.astype({'itemId':'int', 'HC0':'float', 'HC1':'float' })
  # W = W.astype({'userId':'int', 'WC0':'float', 'WC1':'float' })

# print(W)
# print(H)

V = train_data
# print(V)
WH = mult_WH(W, H, V, k)
# print(WH)
# print(type(WH))

R = sub_V_WH(WH)
# print(R)

# print("start calculation of rmse... ")
rmse = rmse_WH_V(WH)
print(f"RMSE: {rmse}")
# print("calculation of rmse end... ")


rank: 	 10 
lamda: 	 60 

944, 1683
RMSE: 3.6443983245418243


In [22]:
# print(R)
# print(W)
# print(H)
# print(WH)
iteration = 10
for i in range(iteration):
  print(f"Iteration: {i}")
  print(datetime.datetime.now())

  W, H, R = ccd_iteration(W, H, R, k)
  # sys.exit()
  # print(W)
  # print(H)
  # print(R)
  if not (i+1)%1:
    # print("start calculation of new WH for compute rmse... ")
    WH = mult_WH(W, H, V, k)
    # print(WH)
    # print("calculation of new WH end... ")
    # print("start calculation of rmse... ")
    rmse = rmse_WH_V(WH)
    print(f"RMSE: {rmse}")
    # sys.exit()

    # print("calculation of rmse end... ")
    if rmse < 0.5:
      print("Convergenced!")
      break

Iteration: 0
2023-02-19 14:50:39.450421
RMSE: 2.212090629607919
Iteration: 1
2023-02-19 14:50:40.974876
RMSE: 1.21028135914007
Iteration: 2
2023-02-19 14:50:42.633407
RMSE: 1.24323588330429
Iteration: 3
2023-02-19 14:50:44.105042
RMSE: 1.2492657475596498
Iteration: 4
2023-02-19 14:50:45.508076
RMSE: 1.2467705975952643
Iteration: 5
2023-02-19 14:50:47.095063
RMSE: 1.247447477935062
Iteration: 6
2023-02-19 14:50:48.889888
RMSE: 1.2472974895630253
Iteration: 7
2023-02-19 14:50:51.023696
RMSE: 1.2475831356115485
Iteration: 8
2023-02-19 14:50:52.817058
RMSE: 1.2475653482125062
Iteration: 9
2023-02-19 14:50:54.400993
RMSE: 1.2476315967519538


In [20]:
userMax, itemMax = get_matrix_dimension(WH)
print(f"{userMax}, {itemMax}")
# WH.count()
print(WH)
print(W)
print(H)


944, 1683
       userId  itemId  rating        WH      diff
0         196     242       3  2.818468  0.032954
1         305     242       5  3.543207  2.122245
2           6     242       4  3.697821  0.091312
3         234     242       4  3.658840  0.116390
4          63     242       3  3.026163  0.000685
...       ...     ...     ...       ...       ...
99995     863    1679       3  0.130251  8.235459
99996     863    1678       1  0.043417  0.915051
99997     863    1680       2  0.086834  3.660204
99998     896    1681       3  0.144968  8.151206
99999     916    1682       3  0.167007  8.025847

[100000 rows x 5 columns]
     userId       WC0       WC1       WC2       WC3       WC4       WC5  \
0         1  1.047432  0.627536  0.496501  0.444177  0.417587  0.338999   
1         2  0.733472  0.436124  0.361015  0.331992  0.300429  0.264809   
2         3  0.562828  0.341955  0.283408  0.259309  0.234738  0.205221   
3         4  0.597559  0.368498  0.312184  0.287087  0.250029  